### ШАГ 0
Импортируем необходимые модули 

In [1]:
# pip install PrettyTable # устанавливаем модуль для вывода таблиц

In [2]:
import sqlite3 as sql
import csv
from prettytable import PrettyTable

### ШАГ 1
Скачивание таблиц и создание БД  

In [6]:
with sql.connect('Gubanov_transaction_for_HW_3.db') as con:
    cur = con.cursor()
    #tr = cur.execute(f'CREATE TABLE IF NOT EXISTS transaction_bd (TRANSACTION_ID INTEGER NOT NULL,TX_DATETIME NUMERIC NOT NULL,CUSTOMER_ID INTEGER NOT NULL,TERMINAL_ID INTEGER NOT NULL,TX_AMOUNT REAL NOT NULL)')
    customer = cur.execute(f'CREATE TABLE IF NOT EXISTS customer_bd (CLIENT_ID INTEGER NOT NULL,START_DT NUMERIC NOT NULL,END_DT NUMERIC NOT NULL,CLIENT_NAME TEXT NOT NULL,YEAR_BIRTH TEXT NOT NULL)')

In [5]:
with open('/Users/mikhailgubanov/Yandex.Disk.localized/Course_of_data_storage_and_processing_systems/transactions_for_dz2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for row in reader:
        cur.execute('''
            INSERT INTO transaction_bd (TRANSACTION_ID,TX_DATETIME, CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT)
            VALUES (?, ?, ?, ?, ?)
        ''',row) # 

# Сохраняем изменения и закрываем соединение
con.commit()
con.close()

In [7]:
with open('/Users/mikhailgubanov/Yandex.Disk.localized/Course_of_data_storage_and_processing_systems/client_info.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for row in reader:
        cur.execute('''
            INSERT INTO customer_bd (CLIENT_ID,START_DT, END_DT,CLIENT_NAME,YEAR_BIRTH)
            VALUES (?, ?, ?, ?, ?)
        ''',row) # 

# Сохраняем изменения и закрываем соединение
con.commit()
con.close()

OperationalError: table transaction_bd has no column named CLIENT_ID